In [2]:
import subprocess

proc = subprocess.Popen(
    ['echo', 'Hello from the child'],
    stdout=subprocess.PIPE)
out, err = proc.communicate()
print out.decode('utf-8')

Hello from the child



In [4]:
proc = subprocess.Popen(['sleep', '0.3'])
while proc.poll() is None:
    print('Working...')
print 'Exit status', proc.poll()

Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...

In [9]:
from threading import Thread, Lock

class Counter(object):
    def __init__(self):
        self.count = 0
        
    def increment(self, offset):
        self.count += offset
        
class LockingCounter(object):
    def __init__(self):
        self.lock = Lock()
        self.count = 0
        
    def increment(self, offset):
        with self.lock:
            self.count += offset
            
def worker(sensor_index, how_many, counter):
        for _ in range(how_many):
            counter.increment(1)
            
def run_threads(func, how_many, counter):
    threads = []
    for i in range(5):
        args = (i, how_many, counter)
        thread = Thread(target=func, args=args)
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()

# GIL 때문에 한 번에 하나의 쓰레드만 동작하기는 하지만,
# counter.count += offset 이 실제로는 3단계에 걸쳐 이루어지는 operation이기 때문에 아래와 같이 예상보다 작은 값이 나올 수 있음.
how_many = 10**5
counter = Counter()
run_threads(worker, how_many, counter)
print 'Counter should be %d, found %d' % (5 * how_many, counter.count)

# 그래서 아래와 같이 lock을 사용해야 안전하다.
counter = LockingCounter()
run_threads(worker, how_many, counter)
print 'Counter should be %d, found %d' % (5 * how_many, counter.count)

Counter should be 500000, found 198467
Counter should be 500000, found 500000


In [6]:
from Queue import Queue
from threading import Thread
import time

class ClosableQueue(Queue):
    SENTINEL = object()
    
    def close(self):
        self.put(self.SENTINEL)
        
    def __iter__(self):
        while True:
            item = self.get()
            try:
                if item is self.SENTINEL:
                    return
                yield item
            finally:
                print '__iter__ task done'
                self.task_done()
                
class StoppableWorker(Thread):
    def __init__(self, func, in_queue, out_queue):
        super(StoppableWorker, self).__init__()
        self.func = func
        self.in_queue = in_queue
        self.out_queue = out_queue
        self.polled_count = 0
        self.work_done = 0
        
    def run(self):
        for item in self.in_queue:
            result = self.func(item)
            print 'done item consumer function.'
            self.out_queue.put(result)
            print 'done item consuming.'

def download_func(item):
    time.sleep(0.1)
    print "task done."
    
download_queue = ClosableQueue()
done_queue = ClosableQueue()
thread = StoppableWorker(download_func, download_queue, done_queue)
print 'start thread'
thread.start()
print 'start add items'
for _ in range(10):
    print 'item added.'
    download_queue.put(object())
download_queue.close()
print 'wait for download queue'
download_queue.join()
print 'done size: ', done_queue.qsize()

start thread
start add items
item added.
item added.
item added.
item added.
item added.
item added.
item added.
item added.
item added.
item added.
wait for download queue
task done.
done item consumer function.
done item consuming.
__iter__ task done
task done.
done item consumer function.
done item consuming.
__iter__ task done
task done.
done item consumer function.
done item consuming.
__iter__ task done
task done.
done item consumer function.
done item consuming.
__iter__ task done
task done.
done item consumer function.
done item consuming.
__iter__ task done
task done.
done item consumer function.
done item consuming.
__iter__ task done
task done.
done item consumer function.
done item consuming.
__iter__ task done
task done.
done item consumer function.
done item consuming.
__iter__ task done
task done.
done item consumer function.
done item consuming.
__iter__ task done
task done.
done item consumer function.
done item consuming.
__iter__ task done
__iter__ task done
done siz

In [18]:
def minimize():
    print "-- 1"
    current = yield
    print "-- 2", current
    while True:
        print "-- 3"
        value = yield current
        print "-- 4", current, value
        current = min(value, current)
        print "-- 5", current, value

it = minimize()
print it
print "next"
next(it)
print "send 10"
print it.send(10)
print "send 4"
print it.send(4)
print "send 22"
print it.send(22)
print "send -1"
print it.send(-1)

<generator object minimize at 0x1057aca50>
next
-- 1
send 10
-- 2 10
-- 3
10
send 4
-- 4 10 4
-- 5 4 4
-- 3
4
send 22
-- 4 4 22
-- 5 4 22
-- 3
4
send -1
-- 4 4 -1
-- 5 -1 -1
-- 3
-1


In [22]:
from time import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def gcd(pair):
    a, b = pair
    low = min(a, b)
    for i in range(low, 0, -1):
        if a % i == 0 and b % i == 0:
            return i

# serial하게 실행할 경우의 시간
numbers = [(1963309, 2265973), (2030677, 3814172), (1551645, 2229620), (2039045, 2020802)]
start = time()
results = list(map(gcd, numbers))
end = time()
print 'Took %f seconds.' % (end - start)

# thread pool을 사용할 경우
# 시작하고 thread pool과 communication하는 overhead때문에 더 느려지는 경우.
start = time()
pool = ThreadPoolExecutor(max_workers=2)
results = list(pool.map(gcd, numbers))
end = time()
print 'Took %f seconds.' % (end - start)

# process pool을 사용할 경우
# 속도는 빠르지만... input / output 데이터가 부모, 자식 프로세스간에 전송될 때마다 serialization / deserialization이 수행되므로, cost가 크다.
# 이 경우처럼, input / output 데이터는 사이즈가 적지만, 계산량이 많은 경우에 적합.
start = time()
pool = ProcessPoolExecutor(max_workers=2)
results = list(pool.map(gcd, numbers))
end = time()
print 'Took %f seconds.' % (end - start)

Took 0.573118 seconds.
Took 0.588674 seconds.
Took 0.359366 seconds.
